# 13.1 Bitly Data from 1.USA.gov

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = './datasets/bitly_usagov/example.txt'

import json

with open(path) as f:
	records = [json.loads(line) for line in f]

In [ ]:
records[0]

In [ ]:
# Counting Time Zones in Pure Python

In [ ]:
# Extract timezones using list comprehension

time_zones = [rec['tz'] for rec in records if 'tz' in rec]

time_zones[:10]

# Counts the timezone 
def get_counts(sequence):
	counts = {}
	for x in sequence:
		if x in counts:
			counts[x] += 1
		else:
			counts[x] = 1
	return counts

# Using more advanced python standard library
from collections import defaultdict

def get_counts2(sequence):
	counts = defaultdict(int) # Values will initialized to 0
	for x in sequence:
		counts[x] += 1
	return counts

counts = get_counts(time_zones)

In [ ]:
# Return top n counts 
def top_counts(count_dict, n=10):
	value_key_pairs = [(count, tz) for tz, count, in count_dict.items()]
	value_key_pairs.sort()
	return value_key_pairs[-n:]

In [ ]:
top_counts(get_counts(time_zones), 5)

In [ ]:
# use Python standard library
from collections import Counter

counts = Counter(time_zones)
counts.most_common(10)

## Counting Time Zones with pandas

In [ ]:
frame = pd.DataFrame(records)

In [ ]:
frame['tz'].head()

In [ ]:
tz_counts = frame['tz'].value_counts()

In [ ]:
tz_counts.head()

In [ ]:
# visualize the data using matplotlib

# make the plots looks nicer by filling a substitute value for unknown value

clean_tz = frame['tz'].fillna('Missing')

clean_tz[clean_tz == ""] = "Unknown"

tz_counts = clean_tz.value_counts()

In [ ]:
import seaborn as sns

subset = tz_counts.head()

sns.barplot(y = subset.index, x = subset.to_numpy())

In [ ]:
# Split off the first token in the string and make another summary of the user behavior

results = pd.Series([x.split()[0] for x in frame['a'].dropna()])

In [ ]:
results.value_counts().head(8)

In [ ]:
# Decompose the top timezones into Windows and non-Window users

# Exclude missing agents from the data
cframe = frame[frame['a'].notna()].copy()

cframe['os'] = np.where(cframe['a'].str.contains('Windows'), 'Windows', 'Not Windows')

cframe['os'].head()

# Group the data by timezone column and list of os
by_tz_os = cframe.groupby(['tz', 'os'])

In [ ]:
agg_counts = by_tz_os.size().unstack().fillna(0)

In [ ]:
# select the top overall timezones

# Construct an indirect index array from the row counts in agg_counts
# After computing row counts with agg_counts.sum('columns'), use argosort() to obtain and index array
indexer = agg_counts.sum('columns').argsort()

In [ ]:
# use `take` to select the rows in the order. Slice off the last 10 rows (largest values)
count_subset = agg_counts.take(indexer[-10:])

count_subset

# Use n-largest to achieve the same result
agg_counts.sum(axis='columns').nlargest(10)

In [ ]:
# To plot in a grouped bar plot

# First stack and reset the index to rearrange the data for better compatibility 
count_subset = count_subset.unstack()

count_subset.name = 'total'

count_subset = count_subset.reset_index()

# sns.barplot(x = 'total', y = 'tz', hue = 'os', data=count_subset)

# Normalize the group percentage sum to 1

def normal_total(group):
	group['normed_total'] = group['total'] / group['total'].sum()
	return group

results = count_subset.groupby('tz').apply(normal_total)

sns.barplot(x = 'normed_total', y='tz', hue='os', data=results)

In [ ]:
# Compute normalized sum more efficiently using trans from groupby method

g = count_subset.groupby('tz')
results2 = count_subset['total'] / g['total'].transform['sum']

In [ ]:
count_subset

# 13.2 MovieLens 1M Dataset


In [ ]:
unames = ["user_id", "gender", "age", "occupation", "zip"]

users = pd.read_table(
    "datasets/movielens/users.dat", sep="::", header=None, names=unames, engine="python"
)

rnames = ["user_id", "movie_id", "rating", "timestamp"]

ratings = pd.read_table(
    "datasets/movielens/ratings.dat",
    sep="::",
    header=None,
    names=rnames,
    engine="python",
)

mnames = ["movie_id", "title", "genres"]
movies = pd.read_table(
    "datasets/movielens/movies.dat",
    sep="::",
    header=None,
    names=mnames,
    engine="python",
)


In [ ]:
users.head(5)

In [ ]:
ratings.head()

In [ ]:
# Merge 'ratings' with 'users' and then merge that result with 'movies' data
data = pd.merge(pd.merge(ratings, users), movies)

In [ ]:
data.iloc[0]

In [ ]:
# To get mean movie ratings for each film grouped by gender

mean_ratings = data.pivot_table('rating', index='title', columns='gender', aggfunc='mean')
mean_ratings.head()

In [ ]:
# Filter down to movies that received at least 250 ratings

ratings_by_title = data.groupby('title').size()

active_titles = ratings_by_title.index[ratings_by_title >= 250]

In [ ]:
active_titles

In [ ]:
# Select rows from mean_ratings

mean_ratings = mean_ratings.loc[active_titles]

In [ ]:
# To see the top films among female viewers, sort F column in descending order

top_female_ratings = mean_ratings.sort_values('F', ascending=False)

top_female_ratings.head()

## Measure Rating Disagreement
Find movies that are most divisive between male and female, add a column to mean_ratings contains the differences in means

In [ ]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

sorted_by_diff = mean_ratings.sort_values('diff')

In [ ]:
sorted_by_diff.head()

In [ ]:
# Find out movies that preferred by men that women didn't rated as highly
sorted_by_diff[::-1].head()

In [ ]:
# get the movies that elicited the most disagreement among viewers

# Disagreement can be measured by the variance of standard deviation

rating_std_by_title = data.groupby('title')['rating'].std()
rating_std_by_title = rating_std_by_title.loc[active_titles]

# Sort in descrending order and select the first 10 rows
rating_std_by_title.sort_values(ascending=False)[:10]

In [ ]:
# use `explode` method to group genres better
movies['genres'].head()

In [ ]:
movies['genres'].head().str.split('|')

In [ ]:
# Split the genres string into a list of genres 
movies['genre'] = movies.pop('genres').str.split('|')

# Calling `explode` method to generate a new DataFrame with one row for each 'inner' element
movies_exploded = movies.explode('genre')
movies_exploded[:10]

In [ ]:
# Merge all three tables together and group by genre

rating_with_genre = pd.merge(pd.merge(movies_exploded, ratings),users)

In [ ]:
genre_ratings = (rating_with_genre.groupby(['genre', 'age'])['rating'].mean().unstack('age'))
genre_ratings

# 13.3 US Baby Names 1880 - 2010 

In [ ]:
# Use Unix syntax to loot at first 10 lines of a file
!head -n 10 'datasets/babynames/yob1880.txt'

In [ ]:
names1880 = pd.read_csv('datasets/babynames/yob1880.txt', names = ['name', 'sex', 'births'])


In [ ]:
# calculate total number of both each year
names1880.groupby('sex').sum()

In [ ]:
# Assemble all of the data into a single DataFrame 

pieces = []

for year in range(1880, 2011):
	path = f'datasets/babynames/yob{year}.txt'
	frame = pd.read_csv(path, names=['name', 'sex', 'births'])

	# Add a column of year
	frame['year'] = year
	pieces.append(frame)

In [ ]:
# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)

In [ ]:
total_births = names.pivot_table('births', index='year', columns='sex', aggfunc=sum)

In [ ]:
total_births.tail()

In [ ]:
total_births.plot(title='Total births by sex and year')

In [ ]:
# Insert a column prop with the fraction of babies given each name relative to the total number of birth
# How popular the name is

def add_prop(group):
	group['prop'] = group['births'] / group['births'].sum()
	return group

names = names.groupby(['year', 'sex']).apply(add_prop)

In [ ]:
names

In [ ]:
# Sanity check to verify the prop will sum to 1
names.groupby(['year', 'sex'])['prop'].sum()

In [ ]:
# Extract a subset of the data to facilitate further analysis
# Top 1000 names for each sex/year combination

def get_top1000(group):
	return group.sort_values('births', ascending=False)[:1000]

grouped = names.groupby(['year', 'sex'])

top1000 = grouped.apply(get_top1000)

In [ ]:
top1000.head()

# drop the group index 
top1000 = top1000.reset_index(drop=True)

## Analyzing Naming Trends

In [ ]:
boys = top1000[top1000['sex']=='M']

girls = top1000[top1000['sex']=='F']

total_births = top1000.pivot_table('births', index='year', columns='name', aggfunc=sum)

In [ ]:
total_births.info()

In [ ]:
# Plot names trends with DataFrame's plot method

subset = total_births[['John', 'Harry', 'Mary', 'Marilyn']]

subset.plot(subplots=True, figsize = (12, 10), title = 'Number of births per year')

## Measuing the increase in naming diversity


In [ ]:
# Proportion of births represented by top 1000 most popular names

table = top1000.pivot_table("prop", index="year", columns="sex", aggfunc=sum)

table.plot(title='Sum of table 1000.prop by year & sex', yticks=np.linspace(0, 1,2, 13))

In [ ]:
# Number of distinct name taken in order of popularity from highest ot lowest]
# boy names from 2010

df = boys[boys['year'] == 2010]

df

In [ ]:
# Calculate how many most popular name it takes to reach 50% 

prop_cumsum = df['prop'].sort_values(ascending=False).cumsum()

In [ ]:
prop_cumsum[:10]

In [ ]:
# searchsorted method return the cumulative sum at provided value
prop_cumsum.searchsorted(0.5)

In [ ]:
df = boys[boys['year'] == 1900]

in1900 = df.sort_values('prop', ascending=False).prop.cumsum()

in1900.searchsorted(0.5) + 1

In [ ]:
# Apply this operation to each year/sex combination

def get_quantile_count(group, q=0.5):
	group = group.sort_values('prop', ascending=False)
	return group.prop.cumsum().searchsorted(q) + 1

diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)

diversity.unstack()

In [ ]:
diversity.unstack().plot.line(title='Number of popular names in top 50%')

## The 'last letter' revolution

To see how the distribution of boy names' final letter has chaged.

In [ ]:
def get_last_letter(x):
    return x[-1]


last_letters = names["name"].map(get_last_letter)

last_letters.name = "last_letter"

table = names.pivot_table(
    "births", index=last_letters, columns=["sex", "year"], aggfunc=sum
)


In [ ]:
subtable = table.reindex(columns=[1910, 1960, 2010], level='year')

subtable.head()

In [ ]:
subtable.sum()

In [ ]:
letter_prop = subtable / subtable.sum()

In [ ]:
letter_prop

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(10, 8))

letter_prop["M"].plot(kind='bar', rot=0, ax=axes[0], title='Male')
letter_prop["F"].plot(kind="bar", rot=0, ax=axes[1], title='Female', legend=False)

In [ ]:
# Normalize by year and sex and select a subset of letter for the boy names
# transposing to make each column a time series

letter_prop = table / table.sum()

In [ ]:
dny_ts = letter_prop.loc[['d', 'n', 'y'], 'M'].T

dny_ts.head()

In [ ]:
dny_ts.plot()

### Boy names that become girl names

In [ ]:
all_names = pd.Series(top1000['name'].unique())

lesly_like = all_names[all_names.str.contains('Lesl')]

In [ ]:
filtered = top1000[top1000['name'].isin(lesly_like)]

filtered.groupby('name')['births'].sum()

In [ ]:
# aggregate by sex and year, normalize within year

table = filtered.pivot_table("births", index="year", columns="sex", aggfunc="sum")

table = table.div(table.sum(axis='columns'), axis='index')

table.tail()

In [ ]:
table.plot(style={'M':'k-', 'F':'k--'})

# 13.4 USDA Food Database

In [ ]:
# load the file into python
import json

db = json.load(open("./datasets/usda_food/database.json"))

len(db)

In [ ]:
db[0].keys()

In [ ]:
db[0]['nutrients'][0]

In [ ]:
nutrients = pd.DataFrame(db[0]['nutrients'])

In [ ]:
nutrients.head()

In [ ]:
# Converting a list of dictionaries to a DataFrame

info_keys = ['description', 'group', 'id', 'manufacturer']

info = pd.DataFrame(db, columns=info_keys)

info.head()

In [ ]:
info.info()

In [ ]:
pd.value_counts(info['group'])[:10]

In [ ]:
# analyse on all of the nutrient data
# Assemble the nutrients for each food into a single large table

# Convert each list of food nutrients to a dataFrame
# add a column for food it
# concatenate the data

nutrients = []

for rec in db:
	fnuts = pd.DataFrame(rec['nutrients'])
	fnuts['id'] = rec['id']
	nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)


In [ ]:
nutrients.head()

In [ ]:
# Check and drop the duplicates

nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {"description": "food", "group": "fgroup"}

info = info.rename(columns=col_mapping, copy=False)

In [ ]:
info.info()

In [ ]:
col_mapping = {'description':'nutrient', 'group':'nutgroup'}

nutrients = nutrients.rename(columns=col_mapping, copy=False)

nutrients

In [ ]:
# Merge info into nutrients
ndata = pd.merge(nutrients, info, on='id')

In [ ]:
ndata.info()

In [ ]:
ndata.iloc[:30000]

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)

result['Zinc, Zn'].sort_values().plot(kind='barh')

In [ ]:
result['Zinc, Zn'].sort_values()

In [ ]:
# Use idxmax or argmax Series method to find which food is most dense in each nutrient

by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

def get_maximum(x):
	return x.loc[x.value.idxmax()]

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]
max_foods.loc['Amino Acids']['food']

# 13.5 2012 Federal Election Commission Database

In [ ]:
import pandas as pd

fec = pd.read_csv('./datasets/fec/P00000001-ALL.csv', low_memory=False)

In [ ]:
fec.info()

In [ ]:
fec.iloc[123456]

In [ ]:
# Ger the unique candidates
unique_cands = fec['cand_nm'].unique()
unique_cands

In [ ]:
# Indicate party affiliation using dictionary
parties = {
  "Bachmann, Michelle": "Republican",
  "Cain, Herman": "Republican",
  "Gingrich, Newt": "Republican",
  "Huntsman, Jon": "Republican",
  "Johnson, Gary Earl": "Republican",
  "McCotter, Thaddeus G": "Republican",
  "Obama, Barack": "Democratic",
  "Pawlenty, Timothy": "Republican",
  "Paul, Ron": "Republican",
  "Perry, Rick": "Republican",
  "Roemer, Charles E. 'Buddy' III": "Republican",
  "Romney, Mitt": "Republican",
  "Santorum, Rick": "Republican"
}

In [ ]:
# Use map method to compute the parties by candidate name:
fec['cand_nm'][123456:124361].map(parties)

In [ ]:
fec['party'] = fec['cand_nm'].map(parties)

In [ ]:
fec['party'].value_counts()

In [12]:
# Restrict the dataset to positive contributions
fec = fec[fec['contb_receipt_amt'] > 0]

In [14]:
# Create a sub set for the two main candidates
fec_mrbo = fec[fec['cand_nm'].isin(['Obama, Barack', 'Romney, Mitt'])]

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,party
411,C00431171,P80003353,"Romney, Mitt","ELDERBAUM, WILLIAM",DPO,AA,340230183,US GOVERNMENT,FOREIGN SERVICE OFFICER,25.0,01-FEB-12,NaN,NaN,NaN,SA17A,780124,Republican
412,C00431171,P80003353,"Romney, Mitt","ELDERBAUM, WILLIAM",DPO,AA,340230183,US GOVERNMENT,FOREIGN SERVICE OFFICER,110.0,01-FEB-12,NaN,NaN,NaN,SA17A,780124,Republican
413,C00431171,P80003353,"Romney, Mitt","CARLSEN, RICHARD",APO,AE,091280020,DEFENSE INTELLIGENCE AGENCY,INTELLIGENCE ANALYST,250.0,13-APR-12,NaN,NaN,NaN,SA17A,785689,Republican
414,C00431171,P80003353,"Romney, Mitt","DELUCA, PIERRE",APO,AE,091280005,CISCO,ENGINEER,30.0,21-AUG-11,NaN,NaN,NaN,SA17A,760261,Republican
415,C00431171,P80003353,"Romney, Mitt","SARGENT, MICHAEL",APO,AE,090120092,RAYTHEON TECHNICAL SERVICES CORP,COMPUTER SYSTEMS ENGINEER,100.0,07-MAR-12,NaN,NaN,NaN,SA17A,780128,Republican


## Donation statistics by Occupation and Employer

In [23]:
# total number of donations by occupation
pd.options.display.max_rows = 100
fec['contbr_occupation'].value_counts()[:100]

RETIRED                                   233990
INFORMATION REQUESTED                      35107
ATTORNEY                                   34286
HOMEMAKER                                  29931
PHYSICIAN                                  23432
INFORMATION REQUESTED PER BEST EFFORTS     21138
ENGINEER                                   14334
TEACHER                                    13990
CONSULTANT                                 13273
PROFESSOR                                  12555
NOT EMPLOYED                                9828
SALES                                       8333
LAWYER                                      8283
MANAGER                                     8024
PRESIDENT                                   7758
STUDENT                                     7071
OWNER                                       6343
EXECUTIVE                                   5506
SELF-EMPLOYED                               5472
WRITER                                      5128
SOFTWARE ENGINEER   

In [ ]:
occ_mapping = {'INFORMATION REQUESTED':'NOT PROVIDED', ''}